In [1]:
import sqlite3
import re
import pandas as pd
import numpy as np

## Clean up subject

In [2]:
# conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
# cur = conn.cursor()
# cur.execute('pragma table_info(Emails)')
# f_1 = 'Id'
# f_2 = 'MetadataTo'
# f_3 = 'MetadataFrom'
# f_4 = 'ExtractedSubject'
# f_5 = 'ExtractedDateSent'
# f_6 = 'ExtractedBodyText'
# f_7 = 'RawText'
# cur.execute('select {c1},{c2},{c3},{c4},{c5},{c6},{c7} from Emails'.\
#             format(c1=f_1,c2=f_2,c3=f_3,c4=f_4,c5=f_5,c6=f_6,c7=f_7))
# cur.fetchall()
# cur.fetchone()
# columns_less = ['id','to','from','sub','date','body','raw']
# columns = [str(tpl[1]) for tpl in columns]
def clean_subject(sub):
    email_type = None
    if re.match('^[Ff][Ww]',sub)!=None:
        email_type='fw'
        sub = re.sub('^[Ff][Ww].*:\s*(.+)',r'\1',sub)
    elif re.match('^[Rr][Ee]:',sub)!=None:
        email_type='re'
        sub = re.sub('^[Rr][Ee]:\s*(.+)',r'\1',sub)
    elif re.match('^[Ff][Vv][Vv]:',sub)!=None:
        email_type='fvv'
        sub = re.sub('^[Ff][Vv][Vv]:\s*(.+)',r'\1',sub)
    return email_type,sub

In [3]:
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, e.SenderPersonId Sender_Id
, e.MetadataTo Receiver, a.PersonId Receiver_Id, e.ExtractedDateSent, e.ExtractedSubject
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id 
left outer join Aliases a on lower(e.MetadataTo)=a.Alias
"""
emails = pd.read_sql_query(sql,conn)
temp = emails.ExtractedSubject.apply(clean_subject)
emails['ExtractedSubject'] = [tpl[1] for tpl in temp]
emails['email_type'] = email_type = [tpl[0] for tpl in temp]
emails.head(20)

,Id,Sender,Sender_Id,Receiver,Receiver_Id,ExtractedDateSent,ExtractedSubject,email_type
0,1,Jake Sullivan,87,H,80.0,"Wednesday, September 12, 2012 10:16 AM",Wow,fw
1,3,Cheryl Mills,32,;H,NaN,"Wednesday, September 12, 2012 11:52 AM",Chris Stevens,re
2,4,Cheryl Mills,32,H,80.0,"Wednesday, September 12,2012 12:44 PM",Cairo Condemnation - Final,fvv
3,5,Hillary Clinton,80,"Abedin, Huma",NaN,,,None
4,6,Hillary Clinton,80,Russorv@state.gov,185.0,"Wednesday, September 12, 2012 01:00 PM",Meet The Right Wing Extremist Behind Anti-Musl...,None
5,7,Cheryl Mills,32,H,80.0,"Wednesday, September 12, 2012 4:00 PM","Anti-Muslim film director in hiding, following...",fw
6,8,Hillary Clinton,80,"Abedin, Huma",NaN,,,None
7,9,Jake Sullivan,87,H,80.0,"Wednesday, September 12, 2012 6:08 PM",Secretary's remarks,fvv
8,11,Jake Sullivan,87,H,80.0,"Sunday, March 13, 2011 10:55 AM",AbZ and Hb3 on Libya and West Bank/Gaza,None
9,13,Jake Sullivan,87,H,80.0,"Wednesday, September 12 2012 700 PM",hey,None


## Clean-up sender receiver

In [4]:
# Fetch data in pandas dataframe
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
# sql = "select * from Emails"
# emails = pd.read_sql_query(sql,conn)
# columns = ['Id','SenderPersonId','MetadataFrom','MetadataTo','ExtractedDateSent','ExtractedSubject','ExtractedBodyText']
# emails[columns].head(5)
# emails[emails['SenderPersonId']==80][columns].head(5)
# x = emails.groupby('SenderPersonId').size()
# print 'No of mails sent by HC =',x[80]

# Assign receiver Id
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.MetadataTo Receiver, a.PersonId Receiver_Id, e.ExtractedDateSent
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id 
left outer join Aliases a on lower(e.MetadataTo)=a.Alias
"""
emails = pd.read_sql_query(sql,conn)
# emails_noNA = emails.dropna(how='any')
# emails.head(20)
idx_NaN = [i for i,val in enumerate(emails['Receiver_Id']) if np.isnan(val)]


# Create dictionary personId:name
sql = "select * from Persons"
persons = pd.read_sql_query(sql,conn)
persons_dict = {}
for entry in persons.itertuples():
    persons_dict[entry[2]]=entry[1]
    
# Fill up missing Receiver_Id fields
receiver_dict = {}
pattern = re.compile('^(\w+),\sp*(\w+)')
for idx in idx_NaN:
    name = emails['Receiver'][idx]
    pat_obj = pattern.match(name)
    if pat_obj!=None:
        person_name = pat_obj.group(2)+' '+pat_obj.group(1)
        try:
            person_id = persons_dict[person_name]
            emails['Receiver_Id'][idx]=person_id
        except:
            pass
temp = emails.shape[0]
emails = emails.dropna(how='any')
print('Discarded emails: ',temp-emails.shape[0])

emails.head(20)

C:\Users\NimishaAg\Anaconda3\lib\site-packages\ipykernel\__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Discarded emails:  174


,Id,Sender,Sender_Id,Receiver,Receiver_Id,ExtractedDateSent
0,1,Jake Sullivan,87,H,80.0,"Wednesday, September 12, 2012 10:16 AM"
2,4,Cheryl Mills,32,H,80.0,"Wednesday, September 12,2012 12:44 PM"
3,5,Hillary Clinton,80,"Abedin, Huma",81.0,
4,6,Hillary Clinton,80,Russorv@state.gov,185.0,"Wednesday, September 12, 2012 01:00 PM"
5,7,Cheryl Mills,32,H,80.0,"Wednesday, September 12, 2012 4:00 PM"
6,8,Hillary Clinton,80,"Abedin, Huma",81.0,
7,9,Jake Sullivan,87,H,80.0,"Wednesday, September 12, 2012 6:08 PM"
8,11,Jake Sullivan,87,H,80.0,"Sunday, March 13, 2011 10:55 AM"
9,13,Jake Sullivan,87,H,80.0,"Wednesday, September 12 2012 700 PM"
10,14,Anne-Marie Slaughter,10,H,80.0,


## Clean up date

In [5]:
# txt = 'Thursday, September 30, 2010 9:04 PM'
# txt = 'Thu Sep 17 06:03:43 2009'
def clean_date_aux(ts):
    pat = re.compile('\'?(?P<weekday>\w+),?\s*(?P<month>\w+) (?P<date>\d+),?\s*(?P<year>\d+) (?P<time>.+) (?P<ampm>[A|P]M)')
    pat1 = re.compile('(?P<weekday>.+) (?P<month>.+) (?P<date>.+) (?P<time>.+) (?P<year>.+)')
    if ts=='':
        return np.NAN
    re_obj = None
    try:
        if pat.match(ts)!=None:
            re_obj = pat.match(ts)
        elif pat1.match(ts)!=None:
            re_obj = pat1.match(ts)
        else:
            raise
        # Process date
        month = re_obj.group('month')
        day = re_obj.group('date')
        year = re_obj.group('year')
        time = re_obj.group('time').replace('.',':')
        # Check for numeric value of day and year
        try:
            month = month[:3]
            day = int(day)
            year = int(year)
        except:
            raise
        # Check time string
        if ':' not in time and len(time)>2:
            time = time[:-2]+':'+time[-2:]
        try:
            time = time + ' ' + re_obj.group('ampm')
        except:
            pass
        ts_str = "{} {} {} {}".format(month,day,year,time)
        try:
            pd_ts = pd.to_datetime(ts_str)
            return pd_ts
        except:
            raise
    except:
        return np.NAN

In [6]:
def clean_date(raw_date):
    return clean_date_aux(raw_date)
emails['Date']=emails.ExtractedDateSent.apply(clean_date)
emails.head(10)
# emails = emails.dropna(how='any')
emails.index = emails.Date
emails.sort_index(inplace=True)
mindate,maxdate = emails.index.min(),emails.index.max()
"Date Range from {} to {}".format(mindate.date(),maxdate.date())
emails[:'2011-01'].Id.count()

6257

## Process email body: Topic modelling

In [7]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [8]:
class HillaryEmails(object):
    def __init__(self,doc_vectors):
        self.doc_vectors = doc_vectors
    def __iter__(self):
        for vector in self.doc_vectors:
            yield vector

In [32]:
def tokenize(text):
    text = text.replace("\n"," ")
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]
def do_BoW(tokens):
    return vocab.doc2bow(tokens)
def get_maxwords(vector):
    minimum = int(len(vector)*0.2)
    minimum = min(10,minimum)
    global vocab
    temp = sorted(vector,key=lambda x:x[1],reverse=True)
    topic = [vocab[it[0]] for it in temp[:minimum]]
    return topic
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.ExtractedDateSent, e.ExtractedBodyText
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id
"""
emails = pd.read_sql_query(sql,conn)
#print(emails)
emails['ExtractedDateSent']=emails.ExtractedDateSent.apply(clean_date)
#print(emails['ExtractedDateSent'])
# Step 1: Mails from Hillary
#emails = emails[emails['Sender_Id']==80]
#print(emails)
# Step 2: Tokenize email body
email_tokens = emails.ExtractedBodyText.apply(tokenize)
#print(email_tokens)
doc_stream = [tokens for tokens in email_tokens]
#print(doc_stream)
print(len(doc_stream))
# Step 3: Build Vocabulary
%time vocab = gensim.corpora.Dictionary(doc_stream)
#print(vocab)
    # print vocab.items()
    # vocab.filter_extremes(no_below=20,no_above=0.8)
    # vocab.filter_extremes(no_above=0.5)
# Step 4: Run BoW on docs
vectors_doc = email_tokens.apply(do_BoW)
print(vectors_doc[3])
maxwords = vectors_doc.apply(get_maxwords)
#print("max\n",maxwords)
#Step 5: save doc vectors in mm file
hillary_emails_corpus = HillaryEmails(vectors_doc)
#print(hillary_emails_corpus)
%time gensim.corpora.MmCorpus.serialize('emails_bow.mm',hillary_emails_corpus)

INFO : adding document #0 to Dictionary(0 unique tokens: [])


7788


INFO : built Dictionary(26189 unique tokens: ['automotive', 'creation', 'rectangular', 'sheets', 'acct']...) from 7788 documents (total 301742 corpus positions)
INFO : storing corpus in Matrix Market format to emails_bow.mm
INFO : saving sparse matrix to emails_bow.mm
INFO : PROGRESS: saving document #0


Wall time: 329 ms
[(1, 1), (2, 2), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


INFO : PROGRESS: saving document #1000
INFO : PROGRESS: saving document #2000
INFO : PROGRESS: saving document #3000
INFO : PROGRESS: saving document #4000
INFO : PROGRESS: saving document #5000
INFO : PROGRESS: saving document #6000
INFO : PROGRESS: saving document #7000
INFO : saved 7788x26189 matrix, density=0.099% (201304/203959932)
INFO : saving MmCorpus index to emails_bow.mm.index


Wall time: 316 ms


In [58]:
from gensim import corpora, models, similarities
tfidf = models.TfidfModel(vectors_doc)
#print(tfidf)
corpus_tfidf = tfidf[vectors_doc]
#for doc in corpus_tfidf:
#    print(doc)
model = models.LdaModel(corpus_tfidf, id2word=vocab, num_topics=100)
corpus_lda = model[corpus_tfidf]
from pprint import pprint
#pprint(model.print_topics(-1))
pprint(model.get_document_topics(vectors_doc[10]))

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : calculating IDF weights for 7788 documents and 26188 features (201304 matrix non-zeros)
INFO : using symmetric alpha at 0.01
INFO : using symmetric eta at 0.01
INFO : using serial LDA version on this node
INFO : running online LDA training, 100 topics, 1 passes over the supplied corpus of 7788 documents, updating model once every 2000 documents, evaluating perplexity every 7788 documents, iterating 50x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #2000/7788
INFO : merging changes from 2000 documents into a model of 7788 documents
INFO : topic #58 (0.010): 0.008*pm + 0.008*pls + 0.007*calling + 0.006*email + 0.006*donna + 0.005*choose + 0.005*messag + 0.005*option + 0.005*lauren + 0.004*send
INFO : topic #42 (0.010): 0.008*libya + 0.008*acting + 0.007*sunday + 0.007*facebooks + 0.006*esther + 0.006*talk + 0.005*sullivanjj + 0.005*gov + 0.005*state + 0.005*aq
INFO : topi

[(11, 0.032399544459246581),
 (15, 0.13140373535887201),
 (48, 0.10800230312143999),
 (50, 0.030049489861467445),
 (51, 0.029140269637242703),
 (58, 0.03951807950664124),
 (60, 0.017918430507101343),
 (68, 0.033181294291386242),
 (69, 0.22188407072877964),
 (76, 0.198957820138216),
 (96, 0.021149013271515055),
 (97, 0.016249481525258784)]


In [63]:
topic = 69
print(model.get_topic_terms(topic, topn=50))
print()
indices = [ind for ind,_ in model.get_topic_terms(topic, topn=50)]
for i in indices:
    print(vocab[i])
print()
print(emails.ExtractedBodyText.get(10))
#print(model.get_term_topics(vocab.token2id('state')))

[(2139, 0.028773432544645601), (2642, 0.015492575342382223), (322, 0.014463027824833148), (796, 0.014114387678011618), (1163, 0.012637920628845355), (156, 0.0092520014076107431), (3957, 0.0092361998007042413), (248, 0.0091865123111745418), (312, 0.0068661560572628941), (409, 0.0059183739295926158), (357, 0.0057744852755316298), (6177, 0.0043459404342122716), (874, 0.0043259935966280675), (1747, 0.0040682952382727006), (378, 0.0040361981132405519), (215, 0.0040118835243991065), (7968, 0.0038322089963970609), (5660, 0.0037777170684219134), (87, 0.0032040389555421504), (179, 0.003171790585571618), (78, 0.0031316601717658165), (4885, 0.0029148626094638568), (170, 0.0027108328227575594), (233, 0.0025102084648676659), (400, 0.0024329232985438329), (139, 0.002426084361283035), (224, 0.0024112756454794004), (20351, 0.0023754912472413198), (2745, 0.0023541155670572926), (371, 0.0023341244871886175), (123, 0.0023025929567070044), (4080, 0.0022622092382931572), (355, 0.002258219825829065), (135, 

In [13]:
hillary_mm_corpus = gensim.corpora.MmCorpus('emails_bow.mm')
print(hillary_mm_corpus)
%time clipped_corpus = gensim.utils.ClippedCorpus(hillary_emails_corpus,1600)
%time lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=20, id2word=vocab, passes=4)

INFO : loaded corpus index from emails_bow.mm.index
INFO : initializing corpus reader from emails_bow.mm
INFO : accepted corpus with 1993 documents, 4249 features, 15844 non-zero entries
INFO : using symmetric alpha at 0.05
INFO : using symmetric eta at 0.05
INFO : using serial LDA version on this node


MmCorpus(1993 documents, 4249 features, 15844 non-zero entries)
Wall time: 0 ns


INFO : running online LDA training, 20 topics, 4 passes over the supplied corpus of 1600 documents, updating model once every 1600 documents, evaluating perplexity every 1600 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -20.086 per-word bound, 1113037.4 perplexity estimate based on a held-out corpus of 1600 documents with 14746 words
INFO : PROGRESS: pass 0, at document #1600/1600
INFO : topic #9 (0.050): 0.013*com + 0.011*print + 0.010*hrod + 0.009*clintonemail + 0.008*know + 0.008*state + 0.008*pis + 0.007*pls + 0.007*pm + 0.006*time
INFO : topic #1 (0.050): 0.016*state + 0.011*tomorrow + 0.010*let + 0.009*com + 0.008*hrod + 0.008*gov + 0.007*time + 0.007*clintonemail + 0.007*thx + 0.006*pls
INFO : topic #19 (0.050): 0.026*pls + 0.016*print + 0.016*ok + 0.013*know + 0.008*need + 0.008*let + 0.008*time + 0.007*set + 0.007*send + 0.005*holbrooke
INFO : topic #11 (0.050): 0.017*pls + 0.013*state + 0.011*ok + 0.010*com + 0.010*list + 0.009*let + 0.009*hrod + 0

Wall time: 8.64 s


In [20]:
from pprint import pprint
pprint(lda_model.print_topics(-1))
pprint(lda_model.get_document_topics(vectors_doc[3]))

INFO : topic #0 (0.050): 0.012*pls + 0.011*told + 0.009*state + 0.009*like + 0.009*lewis + 0.008*address + 0.007*plan + 0.007*work + 0.007*thank + 0.007*list
INFO : topic #1 (0.050): 0.012*tomorrow + 0.010*let + 0.008*state + 0.007*thx + 0.006*pls + 0.006*try + 0.006*welcome + 0.006*check + 0.006*fm + 0.006*think
INFO : topic #2 (0.050): 0.024*com + 0.023*clintonemail + 0.023*hrod + 0.013*pm + 0.010*state + 0.010*gov + 0.010*diplomacy + 0.009*sunday + 0.008*need + 0.008*sid
INFO : topic #3 (0.050): 0.011*fyi + 0.010*let + 0.010*discuss + 0.009*today + 0.008*know + 0.008*tomorrow + 0.008*afghanistan + 0.007*email + 0.007*obama + 0.007*british
INFO : topic #4 (0.050): 0.011*thanks + 0.009*speech + 0.008*mtg + 0.008*night + 0.008*said + 0.007*time + 0.007*pls + 0.006*ok + 0.006*sure + 0.006*want
INFO : topic #5 (0.050): 0.018*state + 0.015*gov + 0.014*thx + 0.013*send + 0.010*letter + 0.010*hrod + 0.009*pls + 0.009*fw + 0.009*going + 0.008*pm
INFO : topic #6 (0.050): 0.042*thx + 0.015*pls

[(0,
  '0.012*pls + 0.011*told + 0.009*state + 0.009*like + 0.009*lewis + '
  '0.008*address + 0.007*plan + 0.007*work + 0.007*thank + 0.007*list'),
 (1,
  '0.012*tomorrow + 0.010*let + 0.008*state + 0.007*thx + 0.006*pls + '
  '0.006*try + 0.006*welcome + 0.006*check + 0.006*fm + 0.006*think'),
 (2,
  '0.024*com + 0.023*clintonemail + 0.023*hrod + 0.013*pm + 0.010*state + '
  '0.010*gov + 0.010*diplomacy + 0.009*sunday + 0.008*need + 0.008*sid'),
 (3,
  '0.011*fyi + 0.010*let + 0.010*discuss + 0.009*today + 0.008*know + '
  '0.008*tomorrow + 0.008*afghanistan + 0.007*email + 0.007*obama + '
  '0.007*british'),
 (4,
  '0.011*thanks + 0.009*speech + 0.008*mtg + 0.008*night + 0.008*said + '
  '0.007*time + 0.007*pls + 0.006*ok + 0.006*sure + 0.006*want'),
 (5,
  '0.018*state + 0.015*gov + 0.014*thx + 0.013*send + 0.010*letter + '
  '0.010*hrod + 0.009*pls + 0.009*fw + 0.009*going + 0.008*pm'),
 (6,
  '0.042*thx + 0.015*pls + 0.010*think + 0.009*better + 0.008*left + '
  '0.007*home + 0.0

In [ ]:
%time tfidf_model = gensim.models.TfidfModel(hillary_mm_corpus, id2word=vocab)
%time lsi_model = gensim.models.LsiModel(tfidf_model[hillary_mm_corpus], id2word=vocab, num_topics=200)

In [ ]:
sentence = "And now for something completely different. They refuse to permit us to obtain the refuse permit. The grand jury commented on a number of other topics, among them the Atlanta and Fulton County purchasing departments which it said, ``Are well operated and follow generally accepted practices which inure to the best interest of both governments?''"

In [ ]:
from nltk import pos_tag, word_tokenize
import nltk

In [ ]:
#tokens = sentence.apply(tokenize)
rawtokens = word_tokenize(sentence)
tokens = [token for token in rawtokens if token not in STOPWORDS]
#print(tokens)
tagged_tokens = nltk.pos_tag(tokens)
words = [word for (word,tag) in tagged_tokens if tag.startswith('NN')]
#print(words)
words_arr = []
words_arr.append(words)
print(words_arr)
%time vocab = gensim.corpora.Dictionary(words_arr)
vectors = vocab.doc2bow(words_arr[0])
print(vectors)
corpus = HillaryEmails(vectors)
print(corpus)
#%time gensim.corpora.MmCorpus.serialize('emails_bow.mm',corpus)
clipped = gensim.utils.ClippedCorpus(corpus,1600)
print(clipped)
#%time lda_model = gensim.models.LdaModel(clipped, num_topics=2, id2word=vocab, passes=4)

## Running R from python

In [ ]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
base = importr('base')
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

In [ ]:
packnames = ['ggplot2']
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install)>0:
    utils.install_packages(StrVector(names_to_install))

## Sentiment Analysis on countries

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [ ]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
n_instances = 100
subj_docs = [(sent,'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent,'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
# subj_docs[10],obj_docs[1]
train_docs = subj_docs[:80]+obj_docs[:80]
test_docs = subj_docs[80:100]+obj_docs[80:100]
senti_analyser = SentimentAnalyzer()

In [ ]:
sent = "He is a murderer"
sentiAnalyser = SentimentIntensityAnalyzer()
sentiAnalyser.polarity_scores(sent)

## Find sentiment of emails

In [ ]:
def getSentiment(text):
    pos = 0
    neg = 0
    neu = 0
    if text=='':
        return 0,pos,neg
    global sentiAnalyser
    text = text.replace('\n',' ')
    sentences = sent_tokenize(text)
    sentences = [sent for sent in sentences if len(sent.split())>2]
    for sent in sentences:
        senti = sentiAnalyser.polarity_scores(sent)
        if senti['neu']>=0.9:
            neu += 1
        else:
            pos += senti['pos']
            neg += senti['neg']
    count = len(sentences)-neu
    if count == 0:
        return len(sentences),pos,neg
    return len(sentences),pos/count,neg/count
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.ExtractedDateSent, e.ExtractedBodyText
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id
"""
emails = pd.read_sql_query(sql,conn)
emails['ExtractedDateSent']=emails.ExtractedDateSent.apply(clean_date)
# Step 1: Mails from Hillary
emails = emails[emails['Sender_Id']==80]
# emails = emails.head(5)
# emails.ExtractedBodyText.apply(getSentiment)
print('Total Email count by Hillary: ',emails.Id.count())
sentiAnalyser = SentimentIntensityAnalyzer()
%time sentiments = emails.ExtractedBodyText.apply(getSentiment)
senti_map = zip(emails.Id.values,sentiments.values)
senti_map = sorted(senti_map,key = lambda item:item[1][2],reverse=True)
print(senti_map[:5])

## Find occurance of country name in mails

In [ ]:
import pycountry

In [ ]:
def get_count(text):
    global country_counter
    global countries_new_name
    for indx,country in enumerate(pycountry.countries):
        name = country.name
        if name in countries_new_name.keys():
            name = countries_new_name[name]
        name = '\\b'+name # Ensuring substring start with country name. eg.Indian, Syrian, India
        all_matches = re.finditer(name,text,re.IGNORECASE)
        n_occurrences = len(list(all_matches))
        country_counter[indx] += n_occurrences
country_counter = [0 for country in pycountry.countries]
country_names = [country.name for country in pycountry.countries]
countries_new_name = {'Syrian Arab Republic':'Syria','Russian Federation':'Russia','Iran, Islamic Republic of':\
                      'Iran','Korea, Democratic People\'s Republic of':'Korea','Palestine, State of':'Palestine',\
                      'Venezuela, Bolivarian Republic of':'Venezuela'}
emails.ExtractedBodyText.apply(get_count)
# let's study count for some countries
country2ind = {}
for indx,country in enumerate(pycountry.countries):
    country2ind[country.name]=indx
country_name_count = zip(country_names,country_counter)
country_name_count = sorted(country_name_count,key = lambda(item):item[1],reverse=True)

In [ ]:
%matplotlib inline
df = pd.DataFrame(np.array(country_counter),country_names)
df = df[df[0]>1]
ax = df.plot(kind='bar',legend=False,title='Occurance of Country names')
ax.set_ylabel('Count')

## Correlation between country vs average sentiment

In [ ]:
def get_occurence_ct(text,country):
    name = country.name
    counter = 0
    global countries_new_name
    if name in countries_new_name.keys():
        name = countries_new_name[name]
    name = '\\b'+name # Ensuring substring start with country name. eg.Indian, Syrian, India
    all_matches = re.finditer(name,text,re.IGNORECASE)
    n_occurrences = len(list(all_matches))
    counter += n_occurrences
    abbr_to_ignore = ["RE", "FM", "TV", "AL", "AQ", "LA", "BEN"]
    if country.alpha2 not in abbr_to_ignore:
        name = '\\b'+country.alpha2+'\\b'
        all_matches = re.finditer(name,text)
        n_occurrences = len(list(all_matches))
        counter += n_occurrences
    if country.alpha3 not in abbr_to_ignore:
        name = '\\b'+country.alpha3+'\\b'
        all_matches = re.finditer(name,text)
        n_occurrences = len(list(all_matches))
        counter += n_occurrences
    return counter
def get_country_sentiment(country):
    global sentiments
    global emails
    res_pos,res_neg = 0,0
    counts = emails.ExtractedBodyText.apply(get_occurence_ct,args=(country,))
    sentiments_pos = [tpl[1] for tpl in sentiments.values]
    sentiments_neg = [tpl[2] for tpl in sentiments.values]
    if sum(counts.values)>0:
        res_pos = sum([ct*pos for ct,pos in zip(counts.values,sentiments_pos)])/sum(counts.values)
        res_neg = sum([ct*neg for ct,neg in zip(counts.values,sentiments_neg)])/sum(counts.values)
    return res_pos,res_neg
countries = pd.Series([country for country in pycountry.countries])
senti = countries.apply(get_country_sentiment)

In [ ]:
senti_pos = np.array([i[0] for i in senti])
df = pd.DataFrame(senti_pos,country_names)
df = df[df[0]>0.1]
ax = df.plot(kind='bar',legend=False,title='Hillary\'s Sentiment about countries')
ax.set_ylabel('postivity')

In [ ]:
senti_neg = np.array([i[1] for i in senti])
df = pd.DataFrame(senti_neg,country_names)
df = df[df[0]>0]
ax = df.plot(kind='bar',legend=False,title='Hillary\'s Sentiment about countries')
ax.set_ylabel('negativity')